# BC205-Exercise 1: Introduction and Sequence Analysis
*by Georgios Kousis Tsampazis*

## Identifying Non-mers in a Bacterial Genome

Non-mers are k-mers that don't have a single instance in a greater corpus (e.g. a genome), that is they do not exist in a genome. Search the genome of E. coli for any given 10-mer and report the 10-mers that do not exist in the genome.
### Genome Source

You can download the *E. coli* genome from [here](https://raw.githubusercontent.com/christoforos-nikolaou/BC205/master/files/ecoli.fa) or use the provided code, which will create a `data_tmp` directory containing the genome file.

### Notebook Instructions

1. **Download the Genome:**
   - Use the link above to obtain the genome of *E. coli*, or run the provided code to create a `data_tmp` directory with the genome file.
   
2. **Run non-kmer Function:**
   - Provide a fasta file and a kmer size 

3. **Output:**
   - Returns a list of non-kmers. 

*Recommendation:* run in a conda env 


### Downlaod data
*Required:*
- *requests*
- *tqdm*

In [6]:
pip install tqdm requests

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import requests
from tqdm import tqdm

# Create the data directory if it doesn't exist.
data_dir = "./data_tmp"
os.makedirs(data_dir, exist_ok=True)
files_to_download = {
    "ecoli.fa": "https://raw.githubusercontent.com/christoforos-nikolaou/BC205/master/files/ecoli.fa"
}

def download_file(url, file_path):
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an error for bad status codes.
    
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte blocks

    with open(file_path, 'wb') as file, tqdm(
            total=total_size, unit='B', unit_scale=True, desc=os.path.basename(file_path)
        ) as progress_bar:
        for data in response.iter_content(block_size):
            file.write(data)
            progress_bar.update(len(data))

for filename, url in files_to_download.items():
    file_path = os.path.join(data_dir, filename)
    if not os.path.exists(file_path):
        print(f"Downloading {filename} ...")
        try:
            download_file(url, file_path)
            print(f"{filename} downloaded successfully.")
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
    else:
        print(f"{filename} already exists. Skipping download.")
print("Files are ready.")


ecoli.fa already exists. Skipping download.
Files are ready.


### Non-kmer Function

In [9]:
import re
import itertools
def hashnonkmers(genomefile, k=10):
    """
    Finds non-kmers with hashing.
    Input is a fasta file !!! Careful Only one sample (only one header instance) and kmer size default is 10
    Example call: hashnonkmers("myfasta.fa", 5) for all non-5-mers in myfasta.fa
    """
    print(f"INFO: Analysis Started")
    
    # Step 1: Reading and Cleaning the Genome
    with open(genomefile, 'r') as file:
        lines = file.readlines()
    seq_lines = [line.strip() for line in lines[1:]] # Skip the first line (header)
    seq_tmp = ''.join(seq_lines) 

    seq = re.sub("[^AGCT]", "", seq_tmp) # Check for weird dna instances 
    
    if len(seq)!=len(seq_tmp):
        print(f"WARNING: {len(seq_tmp)-len(seq)} Not typical DNA characters were found")
        invalid_chars = re.findall("[^AGCT]", seq_tmp)
        unique_invalid_chars = set(invalid_chars) # print weird dna instances
        print("WARNING: Unique invalid characters found:", unique_invalid_chars)
    
    # Step 2: Extracting k-mers from the Genome
    kmer_set = {seq[i:i+k] for i in range(len(seq)-k+1)} # Find kmers in sequence 
    
    # Step 3: Generating All Possible k-mers
    nucleotides = ['A', 'C', 'G', 'T']
    allkmers = [''.join(p) for p in itertools.product(nucleotides, repeat=k)] # Create all available kmers

    # Step 4: Computing the Non-kmers
    non_kmers=[i for i in allkmers if i not in kmer_set] # Found which non-kmers exist in the input sequence 
    
    if (len(non_kmers)+len(kmer_set))!=len(allkmers): # Control-logic Error 
        print("ERROR: Total possible kmers didn't match the sum of non-kmers and kmers found in the sequence")
        return None
    else:
        print(f"INFO: {len(kmer_set)} kmers found in the input genome")
        print(f"INFO: {len(non_kmers)} non-kmers found which is approximately {len(non_kmers)/len(allkmers)*100:.2f}% of all possible kmers.")
        print(f"INFO: Analysis Completed")
        return non_kmers
    
non_kmers=hashnonkmers("./data_tmp/ecoli.fa")    


INFO: Analysis Started
INFO: 825256 kmers found in the input genome
INFO: 223320 non-kmers found which is approximately 21.30% of all possible kmers.
INFO: Analysis Completed


## Function Results

### Function Output

When running the function on the *E. coli* genome, the following output was produced:

- **INFO:** Analysis Started
- **WARNING:** 100 Not typical DNA characters were found
- **WARNING:** Unique invalid characters found: {'N'}
- **INFO:** 825256 kmers found in the input genome
- **INFO:** 223320 non-kmers found which is approximately 21.30% of all possible kmers
- **INFO:** Analysis Completed



## Complexity Analysis

### Current Approach

- **Step 1: Reading and Cleaning the Genome**  
  The function reads the file and concatenates the sequence, running in **O(n)** time, where *n* is the length of the genome.

- **Step 2: Extracting k-mers from the Genome**  
  It extracts all k-mers (substrings of length *k*) from the genome using a set comprehension. This process also runs in approximately **O(n)** time since it iterates through the sequence once. Stolen from [here](https://nbviewer.org/github/christoforos-nikolaou/BC205/blob/master/Classes/Chapter_02_Sequence_Analysis.html).
- **Step 3: Generating All Possible k-mers**  
  All possible k-mers are generated using `itertools.product`, which produces **4^k** combinations. For *k = 10*, this is **1048576** possible k-mers, so this step is **O(4^k)**. Stolen from [here](https://nbviewer.org/github/christoforos-nikolaou/BC205/blob/master/Classes/Chapter_02_Sequence_Analysis.html).

- **Step 4: Computing the Non-kmers**  
  The function computes the non-kmers as the difference between all possible list k-mers and those found in the genome, this step is also approximately **O(4^k)**.

Overall, the time complexity of the current method is **O(n + 4^k)**, which is efficient since it utilizes hashing.

### Alternative "Reverse Logic" Approach - no hashing

If instead of using hashing, the function could:

1. **Generate all available k-mers (O(4^k)).**
2. **For each k-mer, search through the genome to check if it exists.**

In this reverse logic, each k-mer would be checked against the genome, and a naive search through the genome for a substring can take **O(n)** time per k-mer. This approach would result in a time complexity of **O(4^k * n)**.

For example, with *k = 10*, there would be 1048576 k-mers, and for each one, scanning the genome would be required. This would be significantly less efficient compared to the hash-based approach, making it impossible to run (for k=10 and other big k).
### Conclusion

- **Current Method:**  
  Uses set and hashing logic to extract k-mers efficiently with a total complexity of **O(n + 4^k)**.

- **Reverse Logic Alternative:**  
  Would require checking each of the **4^k** possible k-mers against the genome, resulting in a much higher complexity of **O(4^k * n)**.

The current method is clearly preferable in terms of computational efficiency, especially as the value of *k* increases.

